## ASSESSMENT THREE 

### Step 1 | Setting up the Datasets

In [1]:
# Import Libraries
import pandas as pd                                     
import numpy as np                                      
import scipy.stats as st                                    
import seaborn as sns                                   
import matplotlib.pyplot as plt
import statistics as stats
import math 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from scipy.stats import shapiro
from scipy.stats import anderson, ks_2samp
from scipy.stats import mannwhitneyu
from matplotlib.colors import LinearSegmentedColormap
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

In [ ]:
# Read Datasets into DataFrames
dataset1 = pd.read_csv("dataset1.csv")
dataset2 = pd.read_csv("dataset2.csv")
dataset3 = pd.read_csv("dataset3.csv")

# Print Length of Each Dataset
n = len(dataset1)
print("The sample size is: %d" % n)
n = len(dataset2)
print("The sample size is: %d" % n)
n = len(dataset3)
print("The sample size is: %d" % n)

In [ ]:
# Inspect the Data
print('Dataset 1 Preview: ')
print(dataset1.head())

print('\nDataset 2 Preview: ')
print(dataset2.head())

print('\nDataset 3 Preview: ')
print(dataset3.head())

In [ ]:
# Check for Missing Values
print('Missing Values in Dataset 1: ')
print(dataset1.isnull().sum())

print('\nMissing Values in Dataset 2: ')
print(dataset2.isnull().sum())

print('\nMissing Values in Dataset 3: ')
print(dataset3.isnull().sum())

In [ ]:
# Descriptive Statistics
print('Descriptive Statistics Dataset 1: ')
print(dataset1.describe())

print('\nDescriptive Statistics Dataset 2')
print(dataset2.describe())

print('\nDescriptive Statistics Dataset 3')
print(dataset3.describe())

In [ ]:
# Merge Datasets
merged_data = dataset1.merge(dataset2, on='ID').merge(dataset3, on='ID')

#Preview Merged Data
print('Merged Data Keys: ')
print(merged_data.keys())

print('\nMerged Data Types:')
print(merged_data.dtypes)

n = len(merged_data)
print("\nThe sample size is: %d" % n)

## Analysis 1 | Gislene

## Step 2: Test for Gender Differences in Well-being
2.1 Calculating the Overall Well-being Score

Assuming each well-being indicator is equally important, we need to calculate the mean score for each individual.

In [7]:
# List of well-being columns
wellbeing_cols = ['Optm', 'Usef', 'Relx', 'Intp', 'Engs', 'Dealpr', 
                  'Thcklr', 'Goodme', 'Clsep', 'Conf', 'Mkmind', 
                  'Loved', 'Intthg', 'Cheer']

# Calculating the overall well-being
merged_data['Overall_Wellbeing'] = merged_data[wellbeing_cols].mean(axis=1)


## 2.2 Comparing Well-being Scores by Gender

In [8]:
# Separating data by gender
# Map gender codes to labels for clarity
merged_data['Gender_Label'] = merged_data['gender'].map({0: 'Female', 1: 'Male'})

# Separate well-being scores by gender
female_wellbeing = merged_data[merged_data['Gender_Label'] == 'Female']['Overall_Wellbeing']
male_wellbeing = merged_data[merged_data['Gender_Label'] == 'Male']['Overall_Wellbeing']


## Replaced with Georgia's the Anderson-Darling Test

In [ ]:
# Perform Anderson-Darling test for female and male well-being scores
result_female = anderson(female_wellbeing.sample(10000, random_state=1))
result_male = anderson(male_wellbeing.sample(10000, random_state=1))

print(f"Female group Anderson-Darling statistic: {result_female.statistic}")
print(f"Male group Anderson-Darling statistic: {result_male.statistic}")


## 2.4 Perform Mann-Whitney U Test
Since the data is not normally distributed, we'll use a non-parametric test.

In [ ]:
stat, p_value = mannwhitneyu(female_wellbeing, male_wellbeing)
print(f"Mann-Whitney U p-value: {p_value}")

if p_value < 0.05:
    print("Significant difference in well-being between genders.")
else:
    print("No significant difference in well-being between genders.")


## 2.5 Visualising the Results

In [ ]:
# Set white grid style and change palette
sns.set(style="whitegrid")
plt.style.use("default")
sns.set_palette("Blues_r")

# Boxplot of well-being scores by gender
plt.figure(figsize=(8, 6))
sns.boxplot(x='Gender_Label', y='Overall_Wellbeing', data=merged_data)
plt.title('Overall Well-being Scores by Gender')
plt.xlabel('Gender')
plt.ylabel('Overall Well-being Score')
plt.show()


## Step 3: Analysing Screen Time
3.1 Calculate Total and Average Screen Time

In [12]:
# Screen time columns
screen_time_cols = ['C_wk', 'C_we', 'G_wk', 'G_we', 'S_wk', 'S_we', 'T_wk', 'T_we']

# Total screen time per device
merged_data['Total_Screen_Time'] = merged_data[screen_time_cols].sum(axis=1)

# Average screen time per day
merged_data['Avg_Screen_Time'] = merged_data['Total_Screen_Time'] / 7


## 3.2 Analysing the Screen Time Distribution

In [ ]:
# Set the style and colour palette
sns.set(style="whitegrid")
plt.style.use("default")
sns.set_palette("Blues_r")

# Plot the histograms for total and average screen time
plt.figure(figsize=(12, 6))

# Histogram for Total Screen Time
plt.subplot(1, 2, 1)
sns.histplot(merged_data['Total_Screen_Time'], bins=30, kde=True)
plt.title('Total Screen Time Distribution')
plt.xlabel('Total Screen Time (hours)')
plt.ylabel('Frequency')

# Histogram for Average Screen Time
plt.subplot(1, 2, 2)
sns.histplot(merged_data['Avg_Screen_Time'], bins=30, kde=True)
plt.title('Average Screen Time per Day Distribution')
plt.xlabel('Average Screen Time per Day (hours)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


##Note to Self:
There is Outliers: The right-skewed tails suggest that there are outliers, so individuals with considerably high screen time compared to the majority. This may have implications when trying to understand the effect of screen time on well-being, as high users could drive differences.


##Sample code to identify outliers using IQR:

In [14]:
Q1 = merged_data['Avg_Screen_Time'].quantile(0.25)
Q3 = merged_data['Avg_Screen_Time'].quantile(0.75)
IQR = Q3 - Q1

# Defining outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtering outliers
outliers = merged_data[(merged_data['Avg_Screen_Time'] < lower_bound) | (merged_data['Avg_Screen_Time'] > upper_bound)]
non_outliers = merged_data[(merged_data['Avg_Screen_Time'] >= lower_bound) & (merged_data['Avg_Screen_Time'] <= upper_bound)]


##Analysis with and without Outliers:
Sample code for boxplots without outliers:

In [ ]:
# Set the style and colour palette
sns.set(style="whitegrid")
plt.style.use("default")
sns.set_palette("Blues_r")

# Create two subplots for with and without outliers
plt.figure(figsize=(14, 6))

# Plot average screen time by gender with outliers
plt.subplot(1, 2, 1)
sns.boxplot(x='Gender_Label', y='Avg_Screen_Time', data=merged_data)
plt.title('Average Screen Time by Gender (With Outliers)')
plt.xlabel('Gender')
plt.ylabel('Average Screen Time per Day (hours)')

# Plot average screen time by gender without outliers
plt.subplot(1, 2, 2)
sns.boxplot(x='Gender_Label', y='Avg_Screen_Time', data=non_outliers)
plt.title('Average Screen Time by Gender (Without Outliers)')
plt.xlabel('Gender')
plt.ylabel('Average Screen Time per Day (hours)')

plt.tight_layout()
plt.show()


## 3.3 Comparing Screen Time by Gender and visualising

In [ ]:
# Mann-Whitney U Test on screen time (without outliers)
male_screen_time = non_outliers[non_outliers['Gender_Label'] == 'Male']['Avg_Screen_Time']
female_screen_time = non_outliers[non_outliers['Gender_Label'] == 'Female']['Avg_Screen_Time']

# Perform the test
stat, p_value = mannwhitneyu(male_screen_time, female_screen_time)

print(f"Mann-Whitney U Test Statistic: {stat}")
print(f"P-value: {p_value}")


##visualizing the average screen time without outliers:

In [ ]:
# Calculate mean screen time by gender
mean_screen_time = non_outliers.groupby('Gender_Label')['Avg_Screen_Time'].mean().reset_index()

# Bar plot of mean screen time by gender
plt.figure(figsize=(8, 6))
sns.barplot(x='Gender_Label', y='Avg_Screen_Time', data=mean_screen_time, palette="Blues_r")
plt.title('Average Screen Time by Gender (Without Outliers)')
plt.xlabel('Gender')
plt.ylabel('Average Screen Time per Day (hours)')
plt.show()


## 3.4 Assessing the Impact of Screen Time on Well-being 📊
For determine how screen time affects well-being and see if there are gender differences in how screen time influences overall well-being scores.

Notes: creating scatter plots of average screen time vs. overall well-being scores and Separating the scatter plots by gender to visually identify any patterns or relationships.

Below code for Scatterplot (notes from class)

In [ ]:
# Scatter plot for Screen Time vs. Well-being by Gender
plt.figure(figsize=(14, 6))

# Scatter plot for females
plt.subplot(1, 2, 1)
sns.scatterplot(x='Avg_Screen_Time', y='Overall_Wellbeing', data=non_outliers[non_outliers['Gender_Label'] == 'Female'])
plt.title('Screen Time vs. Well-being (Females)')
plt.xlabel('Average Screen Time (hours)')
plt.ylabel('Overall Well-being Score')

# Scatter plot for males
plt.subplot(1, 2, 2)
sns.scatterplot(x='Avg_Screen_Time', y='Overall_Wellbeing', data=non_outliers[non_outliers['Gender_Label'] == 'Male'])
plt.title('Screen Time vs. Well-being (Males)')
plt.xlabel('Average Screen Time (hours)')
plt.ylabel('Overall Well-being Score')

plt.tight_layout()
plt.show()


## Analyzing Interaction Between Screen Time, Gender, and Well-being

In [ ]:
import statsmodels.formula.api as smf

# Creating interaction term
non_outliers['Gender_Numeric'] = non_outliers['Gender_Label'].map({'Female': 0, 'Male': 1})
non_outliers['ScreenTime_Gender_Interaction'] = non_outliers['Avg_Screen_Time'] * non_outliers['Gender_Numeric']

# Fit linear regression model
model = smf.ols('Overall_Wellbeing ~ Avg_Screen_Time + Gender_Numeric + ScreenTime_Gender_Interaction', data=non_outliers).fit()

# Print summary of the regression model
print(model.summary())


## Step 4: Feature Engineering

In [20]:
# Step 4 Feature Engineering applied to merged_data

# Create interaction term between gender and screen time
merged_data['Gender_Numeric'] = merged_data['Gender_Label'].map({'Female': 0, 'Male': 1})
merged_data['ScreenTime_Gender_Interaction'] = merged_data['Avg_Screen_Time'] * merged_data['Gender_Numeric']

# Identify primary device
screen_time_cols = ['C_wk', 'G_wk', 'S_wk', 'T_wk']
merged_data['Primary_Device'] = merged_data[screen_time_cols].idxmax(axis=1)

# Encode primary device as categorical
primary_device_dummies = pd.get_dummies(merged_data['Primary_Device'], prefix='Primary_Device')
merged_data = pd.concat([merged_data, primary_device_dummies], axis=1)


## Step 5: Build the Predictive Models

5.1 Preparing the Data for Modelling

In [21]:

#In this step, we will define the features and target variables for the predictive model.
 #We will also prepare the data by splitting it into training and testing sets.


# Features for modeling
features = ['Gender_Numeric', 'Total_Screen_Time', 'ScreenTime_Gender_Interaction'] + \
           [col for col in merged_data.columns if 'Primary_Device_' in col]

# Target variable
target = 'Overall_Wellbeing'

# Define X and y
X = merged_data[features]
y = merged_data[target]

# One-hot encode categorical variables (if not already done)
X = pd.get_dummies(X, drop_first=True)

# Split the data into training and testing sets (80% train, 20% test)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



## 5.2 Building a Linear Regression Model

In [ ]:
# Step 5.2: 
# creating a Linear Regression model to predict well-being scores using our selected features.
# also evaluating the model's performance with R-squared and Mean Squared Error (MSE).

# Load the data (as feature engineering has already been performed)
# Note: Replace 'data_with_features.csv' with the actual path to our feature-engineered dataset
data = pd.read_csv('data_with_features.csv')

# Features for modelling
features = ['Gender_Numeric', 'Total_Screen_Time', 'ScreenTime_Gender_Interaction'] + \
           [col for col in data.columns if 'Primary_Device_' in col]

# Target variable
target = 'Overall_Wellbeing'

# Define X and y
X = data[features]
y = data[target]

# One-hot encode categorical variables 
X = pd.get_dummies(X, drop_first=True)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialise the linear regression model
model = LinearRegression()

# Train the model on the training data
model.fit(X_train, y_train)

# Predict well-being scores on the test set
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# Print the results
print(f"R-squared: {r2}")
print(f"Mean Squared Error: {mse}")


## 5.3 Interpreting the Model Coefficients


In [ ]:
# Get model coefficients
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
})

# Add interpretation column
coefficients['Impact'] = coefficients['Coefficient'].apply(lambda x: 'Positive' if x > 0 else 'Negative')

# Sort coefficients by their impact
coefficients = coefficients.sort_values(by='Coefficient', ascending=False)

# Print the coefficients DataFrame
print(coefficients)

# Print the intercept
print(f'Intercept: {model.intercept_}')


## 6 Visualising Actual vs. Predicted Well-being
 see how well our model's predictions match the actual well-being scores.

In [ ]:
# Set the style for dark background
sns.set(style="darkgrid")
plt.style.use("dark_background")

# Plotting Actual vs Predicted Well-being Scores
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)

# Plotting the ideal line for comparison (where Actual = Predicted)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')

# Labels and title
plt.xlabel('Actual Well-being Scores')
plt.ylabel('Predicted Well-being Scores')
plt.title('Actual vs. Predicted Well-being Scores')
plt.show()


## 6.1 Evaluating the Effect of Primary Device
see how the primary device used influences overall well-being.

In [ ]:
# Step 1: Creating Interaction Term 
# Note: remember to make sure to replace 'gender' and 'Total_Screen_Time' with the appropriate column names in the dataset
merged_data['Gender_TotalScreenTime'] = merged_data['gender'] * merged_data['Total_Screen_Time']

# Step 2: Determine the Primary Device Used (based on maximum usage hours)
device_columns = ['C_we', 'C_wk', 'G_we', 'G_wk', 'S_we', 'S_wk', 'T_we', 'T_wk']
merged_data['Primary_Device'] = merged_data[device_columns].idxmax(axis=1)

# Step 3: Create Dummy Variables for the Primary Device
merged_data = pd.get_dummies(merged_data, columns=['Primary_Device'], drop_first=True)

# Step 4: Preparing Features for Regression
# Update the features list to include 'Gender_TotalScreenTime'
features = ['gender', 'Total_Screen_Time', 'Gender_TotalScreenTime'] + \
           [col for col in merged_data.columns if 'Primary_Device_' in col]

# Step 5: Define X and y
X = merged_data[features]
y = merged_data['Overall_Wellbeing']

# Step 6: Split Data into Train and Test Sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 7: Train the Linear Regression Model
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Step 8: Evaluate the Model
y_pred = lr_model.predict(X_test)

# Print Model R-squared and Mean Squared Error
from sklearn.metrics import mean_squared_error, r2_score
print('R-squared:', r2_score(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))

# Step 9: Get Model Coefficients
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lr_model.coef_
})
coefficients['Impact'] = coefficients['Coefficient'].apply(lambda x: 'Positive' if x > 0 else 'Negative')
print(coefficients)

# Print the intercept
print(f'Intercept: {lr_model.intercept_}')


## 6.2 Statistical Analysis of Primary Device Effect

In [ ]:
# Checking the columns present in the DataFrame
print("Available columns in merged_data:")
print(merged_data.columns)


In [ ]:
# Columns representing screen time for each device
device_columns = ['C_we', 'C_wk', 'G_we', 'G_wk', 'S_we', 'S_wk', 'T_we', 'T_wk']

# Create a column that shows the primary device with the maximum usage time for each respondent
merged_data['Primary_Device'] = merged_data[device_columns].idxmax(axis=1)

# Confirm that the column was created successfully
print(merged_data[['ID', 'Primary_Device']].head())


## Perform Kruskal-Wallis test

In [ ]:
from scipy.stats import kruskal

# Extracting well-being scores for each primary device group
primary_devices = merged_data['Primary_Device'].unique()
wellbeing_scores = [merged_data[merged_data['Primary_Device'] == device]['Overall_Wellbeing'] for device in primary_devices]

# Perform Kruskal-Wallis test
stat, p_value = kruskal(*wellbeing_scores)

print(f"Kruskal-Wallis test statistic: {stat}")
print(f"P-value: {p_value}")

# Interpretation
if p_value < 0.05:
    print("There is a significant difference in well-being scores among different primary device groups (Reject H0).")
else:
    print("There is no significant difference in well-being scores among different primary device groups (Fail to reject H0).")


##Visualising the Differences

In [ ]:
# Set style for a dark background
sns.set(style="darkgrid")
plt.style.use("dark_background")

# Boxplot for Well-being Scores by Primary Device
plt.figure(figsize=(10, 6))
sns.boxplot(x='Primary_Device', y='Overall_Wellbeing', data=merged_data)
plt.title('Well-being Scores by Primary Device')
plt.xlabel('Primary Device')
plt.ylabel('Well-being Score')
plt.show()


## 6.3 Interaction Between Gender and Primary Device
Two-Way ANOVA to Check Interaction

In [30]:
#Converting Variables to Categorical
merged_data['Gender_Label'] = merged_data['Gender_Label'].astype('category')
merged_data['Primary_Device'] = merged_data['Primary_Device'].astype('category')


In [ ]:
#Check for Missing or Empty Groups
print(merged_data.groupby(['Gender_Label', 'Primary_Device']).size())


#Recreating Primary_Device Correctly

In [32]:
# Creating the Primary_Device based on max screen time columns
screen_time_cols = ['C_we', 'C_wk', 'G_we', 'G_wk', 'S_we', 'S_wk', 'T_we', 'T_wk']
merged_data['Primary_Device'] = merged_data[screen_time_cols].idxmax(axis=1)
merged_data['Primary_Device'] = merged_data['Primary_Device'].str.extract(r'([A-Z])')


In [ ]:
#rerun the ANOVA
merged_data['Primary_Device'] = merged_data['Primary_Device'].astype('category')

# Running the two-way ANOVA again
import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('Overall_Wellbeing ~ Gender_Label * Primary_Device', data=merged_data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)


## 6.4 Visualising the Interaction Effect

In [ ]:
# Setting the style for dark background
sns.set(style="darkgrid")
plt.style.use("dark_background")

# Plotting interaction between Gender and Primary Device
plt.figure(figsize=(12, 6))
sns.boxplot(x='Primary_Device', y='Overall_Wellbeing', hue='Gender_Label', data=merged_data, palette='Blues_r')
plt.title('Interaction Effect of Gender and Primary Device on Well-being Scores')
plt.xlabel('Primary Device')
plt.ylabel('Overall Well-being Score')
plt.legend(title='Gender')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Analysis 2 | Georgia

### Part A | Ethnic Groups + Well-being Scores

### Step 1 | Data Exploration

In [ ]:
# Define Majority Ethnic Group & Minority Ethnic Group
# Ethnic group labels
merged_data['ethnic_group'] = merged_data['minority'].map({0: 'Majority', 1: 'Minority'})

# Filter the data into majority/minority groups
majority_data = merged_data[merged_data['minority'] == 0]
minority_data = merged_data[merged_data['minority'] == 1]

# Preview Ethnic Group Data
print('Filtered Data - Majority: (%.d rows and %.d columns.)' % (majority_data.shape[0], majority_data.shape[1]))
print('Filtered Data - Minority: (%.d rows and %.d columns.)' % (minority_data.shape[0], minority_data.shape[1]))

print('Majority Group Preview: ')
print(majority_data.head())

print('\nMinority Group Preview: ')
print(minority_data.head())

In [36]:
# Define Well-being
# Well-being Columns
wellbeing = ['Optm', 'Usef', 'Relx', 'Intp', 'Engs', 'Dealpr', 'Thcklr', 'Goodme', 'Clsep', 'Conf', 'Mkmind', 'Loved', 'Intthg', 'Cheer']

# Well-being Mean
merged_data['overall_wellbeing'] = merged_data[wellbeing].mean(axis=1)

# Create DataFrame for majority/minority well-being scores
majority_wellbeing = majority_data[wellbeing]
minority_wellbeing = minority_data[wellbeing]

In [ ]:
# Majority Descriptive Statistics
# Compute means
wellbeing_means_majority = majority_data[wellbeing].mean()

print('Mean Scores for Each Wellbeing Indicator (Majority Group):')
for indicator, mean in wellbeing_means_majority.items():
    print(f'{indicator}: {mean:.2f}')

# Compute overall mean
overall_wellbeing_mean_majority = wellbeing_means_majority.mean()

print('\nOverall Wellbeing Mean (Majority Group): %.2f' % overall_wellbeing_mean_majority)
print()

# Minority Descriptive Statistics
# Compute means
wellbeing_means_minority = minority_data[wellbeing].mean()

print('Mean Scores for Each Wellbeing Indicator (Minority Group):')
for indicator, mean in wellbeing_means_minority.items():
    print(f'{indicator}: {mean:.2f}')

# Compute overall mean
overall_wellbeing_mean_minority = wellbeing_means_minority.mean()

print('\nOverall Wellbeing Mean (Minority Group): %.2f' % overall_wellbeing_mean_minority)

In [ ]:
sns.set()
plt.style.use('default') 

# Data Visualisation | Bar Plots
wellbeing_means_majority = pd.Series([3.26, 3.09, 3.11, 3.32, 3.04, 3.37, 3.48, 3.24, 3.58, 3.27, 3.86, 3.93, 3.44, 3.50], 
                                      index=['Optm', 'Usef', 'Relx', 'Intp', 'Engs', 'Dealpr', 'Thcklr', 'Goodme', 'Clsep', 'Conf', 'Mkmind', 'Loved', 'Intthg', 'Cheer'])

wellbeing_means_minority = pd.Series([3.35, 3.18, 3.05, 3.10, 3.06, 3.37, 3.52, 3.38, 3.47, 3.44, 3.83, 3.81, 3.60, 3.49], 
                                      index=['Optm', 'Usef', 'Relx', 'Intp', 'Engs', 'Dealpr', 'Thcklr', 'Goodme', 'Clsep', 'Conf', 'Mkmind', 'Loved', 'Intthg', 'Cheer'])

# Sub-plots
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Majority Mean Well-Being Scores
axes[0].bar(wellbeing_means_majority.index, wellbeing_means_majority.values, color='#BC5090')
axes[0].set_title('Mean Well-being Score (Majority Group)', fontsize=16, fontweight='bold', color='#BC5090')
axes[0].set_xlabel('Well-being Indicator', fontsize=14, fontweight='bold', color='#BC5090')
axes[0].set_ylabel('Mean Score', fontsize=14, fontweight='bold', color='#BC5090')
axes[0].set_ylim(0, 6)
axes[0].grid(axis='y', linestyle='--', alpha=0.7)
for index, value in enumerate(wellbeing_means_majority.values):
    axes[0].text(index, value + 0.1, f'{value:.2f}', ha='center', fontsize=12,color='#003F5C')

# Minority Mean Well-Being Scores
axes[1].bar(wellbeing_means_minority.index, wellbeing_means_minority.values, color='#58508D')
axes[1].set_title('Mean Well-being Score (Minority Group)', fontsize=16, fontweight='bold', color='#58508D')
axes[1].set_xlabel('Well-being Indicator', fontsize=14, fontweight='bold', color='#58508D')
axes[1].set_ylabel('Mean Score', fontsize=14, fontweight='bold', color='#58508D')
axes[1].set_ylim(0, 6)
axes[1].grid(axis='y', linestyle='--', alpha=0.7)
for index, value in enumerate(wellbeing_means_minority.values):
    axes[1].text(index, value + 0.1, f'{value:.2f}', ha='center', fontsize=12, color='#003F5C')
    
plt.tight_layout()
plt.show()

### Step 2 | Test for Normality

In [ ]:
# Shapiro-Wilk Test
df_majority = majority_wellbeing.sample(5000, random_state=1)
df_minority = minority_wellbeing.sample(5000, random_state=1)

_, p_majority = shapiro(df_majority)
_, p_minority = shapiro(df_minority)

print('Majority Group Normality p-value: ', p_majority) 
print('Minority Group Normality p-value: ', p_minority)
print() 

if p_majority < 0.05:
        print('Majority Group Data is not normally distributed.')
else:
        print('Majority Group Data is normally distributed.')

        
if p_minority < 0.05:
        print('Minority Group Data is not normally distributed.')
else:
        print('Minority Group Data is normally distributed.')

In [ ]:
# Anderson-Darling Test
# Calculate the mean scores for the selected columns for both groups
majority_wellbeing_mean = merged_data[merged_data['minority'] == 0][['Optm', 'Usef', 'Relx', 'Intp', 'Engs', 'Dealpr', 'Thcklr', 'Goodme', 'Clsep', 'Conf', 'Mkmind', 'Loved', 'Intthg', 'Cheer']].mean(axis=1) 
minority_wellbeing_mean = merged_data[merged_data['minority'] == 1][['Optm', 'Usef', 'Relx', 'Intp', 'Engs', 'Dealpr', 'Thcklr', 'Goodme', 'Clsep', 'Conf', 'Mkmind', 'Loved', 'Intthg', 'Cheer']].mean(axis=1) 

# Perform the A-D test on the means
result1 = anderson(majority_wellbeing_mean.dropna())
result2 = anderson(minority_wellbeing_mean.dropna())

# Perform Kolmogorov-Smirnov test to compare samples
ks_statistic, ks_p_value = ks_2samp(majority_wellbeing_mean.dropna(), minority_wellbeing_mean.dropna())
print('\nKS Statistic:', ks_statistic)
print('P-value:', ks_p_value)

if ks_p_value < 0.05:
    print('The distributions of the two groups are significantly different.')
else:
    print('The distributions of the two groups are not significantly different.')

# Print distribution outcome based on A-D test
if result1.statistic > result1.critical_values[2]:
    print('Majority Group Data is not normally distributed.')
else:
    print('Majority Group Data is normally distributed.')

if result2.statistic > result2.critical_values[2]:
    print('Minority Group Data is not normally distributed.')
else:
    print('Minority Group Data is normally distributed.')


### Step 3 | Hypothesis Testing

In [ ]:
# Mann-Whitney U Test
p_values = []
for column in wellbeing: 
        stat, p_val = mannwhitneyu(majority_data[column], minority_data[column])
        p_values.append(p_val)
        print(f"Mann-Whitney U p-value for {column}: ", p_val)


        if p_val < 0.05:
                print(f'There is a statistically significant difference in {column} scores between ethnicity groups.')
        else:
                print(f'There is no statistically significant difference in {column} scores between ethnicity groups.')
        print()        

In [ ]:
# Effect Sizes
def mann_whitney_effect_size(u, n1, n2):
    return (u / (n1 * n2)) - 0.5

n_majority = majority_data.shape[0]
n_minority = minority_data.shape[0]

for column in wellbeing: 
    stat, p_val = mannwhitneyu(majority_data[column], minority_data[column])
    u = stat
    effect_size = mann_whitney_effect_size(u, n_majority, n_minority)
    
    
    if effect_size < 0.1:
        interpretation = 'small'
    elif effect_size < 0.3:
        interpretation = 'medium'
    elif effect_size >= 0.5:
        interpretation = 'large'
    else:
        interpretation = 'between medium and large'
    
    print(f"Effect size for {column}: {effect_size:.3f} ({interpretation})")

In [ ]:
# Data Visualisation | Box Plot
wellbeing_long = merged_data.melt(id_vars='ethnic_group', value_vars=wellbeing, var_name='Wellbeing Indicator', value_name='Score')

plt.figure(figsize=(14, 8))
sns.boxplot(x='Wellbeing Indicator', y='Score', hue='ethnic_group', data=wellbeing_long, palette={'Majority': '#BC5090', 'Minority': '#58508D'})
plt.title('Wellbeing Scores Across Ethnic Groups', fontsize=16, fontweight='bold', color='#003F5C')
plt.xlabel('Wellbeing Indicator', fontsize=14, fontweight='bold', color='#003F5C')
plt.ylabel('Score', fontsize=14, fontweight='bold', color='#003F5C')
plt.xticks(rotation=45, fontsize=12, color='#003F5C')
plt.legend(title='Ethnic Group', title_fontsize='13', fontsize='12')
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.ylim(wellbeing_long['Score'].min() - 0.5, wellbeing_long['Score'].max() + 0.5)
plt.show()


### Part B | Ethnic Groups + Screen Time

### Step 1 | Data Exploration

In [44]:
# Define Screen Time
# Device + Screen Time Columns
screentime = ['C_wk', 'C_we', 'G_wk', 'G_we', 'S_wk', 'S_we', 'T_wk', 'T_we']

# Screen Time Mean
merged_data['overall_screentime'] = merged_data[screentime].mean(axis=1)

# Create DataFrame for majority/minority well-being scores
majority_screentime = majority_data[screentime]
minority_screentime = minority_data[screentime]

In [ ]:
# Majority Descriptive Statistics
# Compute means
screentime_means_majority = majority_data[screentime].mean()

print('Mean Hours for Each Device (Majority Group):')
for indicator, mean in screentime_means_majority.items():
    print(f'{indicator}: {mean:.2f}')

# Compute overall mean
overall_screentime_mean_majority = screentime_means_majority.mean()

print('\nOverall Screen Time Mean (Majority Group): %.2f' % overall_screentime_mean_majority)
print()

# Minority Descriptive Statistics
# Compute means
screentime_means_minority = minority_data[screentime].mean()

print('Mean Hours for Each Device (Minority Group):')
for indicator, mean in screentime_means_minority.items():
    print(f'{indicator}: {mean:.2f}')

# Compute overall mean
overall_screentime_mean_minority = screentime_means_minority.mean()

print('\nOverall Screen Time Mean (Minority Group): %.2f' % overall_screentime_mean_minority)

In [ ]:
# Data Visualisation | Bar Plots
screentime_means_majority = pd.Series([1.71, 2.13, 1.08, 1.80, 2.91, 3.51, 2.56, 3.60], 
                                      index=['C_we', 'C_wk', 'G_we', 'G_wk', 'S_we', 'S_wk', 'T_we', 'T_wk'])

screentime_means_minority = pd.Series([1.95, 2.44, 0.71, 1.47, 2.81, 3.49, 2.53, 3.82], 
                                      index=['C_we', 'C_wk', 'G_we', 'G_wk', 'S_we', 'S_wk', 'T_we', 'T_wk'])

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Majority Mean Screen Time 
axes[0].bar(screentime_means_majority.index, screentime_means_majority.values, color='#BC5090', label='Majority Group')
axes[0].set_title('Majority Ethnic Group Mean Hours of Screen Time', fontsize=16, fontweight='bold', color='#BC5090')
axes[0].set_xlabel('Device', fontsize=14, color='#BC5090', fontweight='bold')
axes[0].set_ylabel('Mean Hours', fontsize=14, color='#BC5090', fontweight='bold')
axes[0].set_ylim(0, 6)
axes[0].grid(axis='y', linestyle='--', alpha=0.7)
for index, value in enumerate(screentime_means_majority.values):
    axes[0].text(index, value + 0.1, f'{value:.2f}', ha='center', fontsize=12, color='#003F5C')

# Minority Mean Screen Time
axes[1].bar(screentime_means_minority.index, screentime_means_minority.values, color='#58508D', label='Minority Group')
axes[1].set_title('Minority Ethnic Group Mean Hours of Screen Time', fontsize=16, fontweight='bold', color='#58508D')
axes[1].set_xlabel('Device', fontsize=14, color='#58508D', fontweight='bold')
axes[1].set_ylabel('Mean Hours', fontsize=14, color='#58508D', fontweight='bold')
axes[1].set_ylim(0, 6)
axes[1].grid(axis='y', linestyle='--', alpha=0.7)
for index, value in enumerate(screentime_means_minority.values):
    axes[1].text(index, value + 0.1, f'{value:.2f}', ha='center', fontsize=12, color='#003F5C')
axes[0].legend()
axes[1].legend()

plt.tight_layout()
plt.show()


### Step 2 | Test for Normality

In [ ]:
# Shapiro-Wilk Test
df_majority = majority_screentime.sample(5000, random_state=1)
df_minority = minority_screentime.sample(5000, random_state=1)

_, p_majority = shapiro(df_majority)
_, p_minority = shapiro(df_minority)

print('Majority Group Normality p-value: ', p_majority) 
print('Minority Group Normality p-value: ', p_minority)
print() 

if p_majority < 0.05:
        print('Majority Group Data is not normally distributed.')
else:
        print('Majority Group Data is normally distributed.')

        
if p_minority < 0.05:
        print('Minority Group Data is not normally distributed.')
else:
        print('Minority Group Data is normally distributed.')

In [ ]:
# Anderson-Darling Test
# Calculate the mean scores for the selected columns for both groups
majority_screentime_mean = merged_data[merged_data['minority'] == 0][['C_wk', 'C_we', 'G_wk', 'G_we', 'S_wk', 'S_we', 'T_wk', 'T_we']].mean(axis=1) 
minority_screentime_mean = merged_data[merged_data['minority'] == 1][['C_wk', 'C_we', 'G_wk', 'G_we', 'S_wk', 'S_we', 'T_wk', 'T_we']].mean(axis=1) 

# Perform the A-D test on the means
result1 = anderson(majority_screentime_mean.dropna())
result2 = anderson(minority_screentime_mean.dropna())

# Perform Kolmogorov-Smirnov test to compare samples
ks_statistic, ks_p_value = ks_2samp(majority_screentime_mean.dropna(), minority_screentime_mean.dropna())
print('\nKS Statistic:', ks_statistic)
print('P-value:', ks_p_value)

if ks_p_value < 0.05:
    print('The distributions of the two groups are significantly different.')
else:
    print('The distributions of the two groups are not significantly different.')

# Print distribution outcome based on A-D test
if result1.statistic > result1.critical_values[2]:
    print('Majority Group Data is not normally distributed.')
else:
    print('Majority Group Data is normally distributed.')

if result2.statistic > result2.critical_values[2]:
    print('Minority Group Data is not normally distributed.')
else:
    print('Minority Group Data is normally distributed.')

### Step 3 | Hypothesis Testing

In [ ]:
# Perform Mann-Whitney U Test
p_values = []
for column in screentime: 
        stat, p_val = mannwhitneyu(majority_data[column], minority_data[column])
        p_values.append(p_val)
        print(f"Mann-Whitney U p-value for {column}: ", p_val)

        if p_val < 0.05:
                print(f'There is a statistically significant difference in {column} hours between ethnicity groups.')
        else:
                print(f'There is no statistically significant difference in {column} hours between ethnicity groups.')
        print()        

In [ ]:
# Effect Sizes
def mann_whitney_effect_size(u, n1, n2):
    return (u / (n1 * n2)) - 0.5

n_majority = majority_data.shape[0]
n_minority = minority_data.shape[0]

for column in screentime: 
    stat, p_val = mannwhitneyu(majority_data[column], minority_data[column])
    u = stat
    effect_size = mann_whitney_effect_size(u, n_majority, n_minority)
    
    
    if effect_size < 0.1:
        interpretation = 'small'
    elif effect_size < 0.3:
        interpretation = 'medium'
    elif effect_size >= 0.5:
        interpretation = 'large'
    else:
        interpretation = 'between medium and large'
    
    print(f"Effect size for {column}: {effect_size:.3f} ({interpretation})")


In [ ]:
# Data Visualisation | Box Plot
# Melt data
screentime_long = merged_data.melt(id_vars='ethnic_group', value_vars=screentime, var_name='Device', value_name='Hours')

plt.figure(figsize=(14, 8))
sns.boxplot(x='Device', y='Hours', hue='ethnic_group', data=screentime_long, palette={'Majority': '#BC5090', 'Minority': '#58508D'})

plt.title('Screen Time Across Ethnic Groups', fontsize=16, fontweight='bold', color='#003F5C')
plt.xlabel('Device', fontsize=14, fontweight='bold', color='#003F5C')
plt.ylabel('Hours', fontsize=14, fontweight='bold', color='#003F5C')
plt.xticks(rotation=45, fontsize=12, color='#003F5C')
plt.legend(title='Ethnic Group', title_fontsize='13', fontsize='12')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.ylim(0, screentime_long['Hours'].max() + 1) 
plt.tight_layout()
plt.show()

### Part C | Predictive Modelling

In [ ]:
# Isolate the Minority Data into DataFrame
minority_data = merged_data[merged_data['minority'] == 1]

n = len(minority_data)
print("The sample size is: %d" % n)

print(minority_data.columns)

# Create new DataFrame
minority_df = minority_data.copy()

minority_df['wellbeing'] = minority_data[['Optm', 'Usef', 'Relx', 'Intp', 'Engs', 'Dealpr', 'Thcklr', 'Goodme', 'Clsep', 'Conf', 'Mkmind', 'Loved', 'Intthg', 'Cheer']].mean(axis=1)

minority_df['screentime'] = minority_data[["C_we", "C_wk", "G_we", "G_wk", "S_we", "S_wk", "T_we", "T_wk"]].mean(axis=1)

In [ ]:
# Sample size
n = len(minority_df)
print(f"The sample size is: {n}")

# Confirm means
wellbeing_mean = minority_df['wellbeing'].mean()
screentime_mean = minority_df['screentime'].mean()

# Print results
print(f"Minority Wellbeing Mean: {wellbeing_mean:.2f}")
print(f"Minority Screen Time Mean: {screentime_mean:.2f}")


In [ ]:
# Calculate the correlation with Pearson's R
correlation_matrix = minority_df[['screentime', 'wellbeing']].corr(method='pearson')

print(correlation_matrix)

# Data Visualisation | Heatmap
colors = ['#BC5090', '#58508D', '#003F5C'] 
custom_cmap = LinearSegmentedColormap.from_list('custom', colors)

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap=custom_cmap, cbar=True, square=True, fmt=".2f")
plt.title('Pearson Correlation Matrix Heatmap', fontsize=16, fontweight='bold', color='#003F5C')
plt.xticks(fontsize=12, color='#003F5C')
plt.yticks(fontsize=12, color='#003F5C')

plt.tight_layout()
plt.show()

In [ ]:
# Data Visualisation | Scatterplot 
sns.lmplot(x='wellbeing', y='screentime', data=minority_df, height=6, aspect=1.25, 
           scatter_kws={'s': 2.5, 'alpha': 0.6}, line_kws={'color': '#BC5090', 'linewidth': 3})

plt.xlabel('Well-being', fontsize=14, fontweight='bold', color='#003F5C')
plt.ylabel('Screen Time', fontsize=14, fontweight='bold', color='#003F5C')
plt.title('Relationship between Screen Time and Well-being (Minority Group)', fontsize=16, fontweight='bold', color='#003F5C')
plt.show()


##### The outcome of this shows there is a very small, negative relationship between screentime and well-being for minority group. Therefore, Linear Regression Model may not be the most appropriate model. 

### Step 9 | Log Transformation

In [ ]:
# Log transformation screen time
minority_df['log_screentime'] = np.log(minority_df['screentime'] + 1)

# Review distribution 
sample = minority_df['log_screentime'].sample(5000, random_state=1)
stat, p_value = shapiro(sample)
if p_value < 0.05:
    print("Data is not normally distributed")
    print(p_value)

In [ ]:
# Log transformation Well-being
minority_df['log_wellbeing'] = np.log(minority_df['wellbeing'] + 1)

# Review distribution
sample = minority_df['log_wellbeing'].sample(5000, random_state=1)
stat, p_value = shapiro(sample)
if p_value < 0.05:
    print("Data is not normally distributed")
    print(p_value)

In [ ]:
# Calculate the correlation with Pearson's R
correlation_matrix = minority_df[['log_screentime', 'log_wellbeing']].corr(method='pearson')

print(correlation_matrix)

# Data Visualisation | Heatmap
colors = ['#BC5090', '#58508D', '#003F5C'] 
custom_cmap = LinearSegmentedColormap.from_list('custom', colors)

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap=custom_cmap, cbar=True, square=True, fmt=".2f")
plt.title('Pearson Correlation Matrix Heatmap', fontsize=16, fontweight='bold', color='#003F5C')
plt.xticks(fontsize=12, color='#003F5C')
plt.yticks(fontsize=12, color='#003F5C')

plt.tight_layout()
plt.show()


In [ ]:
# Data Visualisation | Scatterplot 
sns.lmplot(x='log_wellbeing', y='log_screentime', data=minority_df, height=6, aspect=1.25, 
           scatter_kws={'s': 2.5, 'alpha': 0.6}, line_kws={'color': '#BC5090', 'linewidth': 3})

plt.xlabel('Log Well-being', fontsize=14, fontweight='bold', color='#003F5C')
plt.ylabel('Log Screen Time', fontsize=14, fontweight='bold', color='#003F5C')
plt.title('Relationship between Log Screen Time and Log Well-being (Minority Group)', fontsize=16, fontweight='bold', color='#003F5C')

plt.tight_layout
plt.show()


In [ ]:
# Separate variables
x = minority_df['log_screentime'].values.reshape(-1, 1) 
y = minority_df['log_wellbeing'].values

# Split dataset into training (60%) and testing (40%) 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=0)

# Build LRM
model = LinearRegression()

# Train (fit) the LRM using training set
model.fit(X_train, y_train)

# Print intercept and coefficient from LRM
print("Intercept: ", model.intercept_)
print("Coefficient: ", model.coef_)

# Use LRM to predict values of (y)
y_pred = model.predict(X_test)

# Show predicted values of (y) next to the actual values of (y)
df_pred = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})
print(df_pred)

# Compute standard performance metrics of LRM
mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = math.sqrt(metrics.mean_squared_error(y_test, y_pred))
rmse_norm = rmse / (y.max() - y.min())
r_2 = metrics.r2_score(y_test, y_pred)

print("Multiple Linear P performance:")
print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("RMSE (Normalised): ", rmse_norm)
print("R^2: ", r_2)

In [ ]:
minority_data = {
    'minority_group': ['1'] * 22267,
    'screen_time_minutes': np.random.randint(0, 421, size=22267),
    'wellbeing_score': np.random.rand(22267) * 10 
}
df = pd.DataFrame(minority_data)

# Bin edges and labels
bin_edges = [0, 120, 240, 421]
bin_labels = ['0-2h', '2-4h', '4-7h']

# Create bins for screen time
df['screen_time_binned'] = pd.cut(df['screen_time_minutes'], bins=bin_edges, labels=bin_labels)

# Filter for "4-7h"
subgroup = df[df['screen_time_binned'] == '4-7h']

# Correlation coefficient between screen time and wellbeing score
correlation = subgroup['screen_time_minutes'].corr(subgroup['wellbeing_score'])

print("Correlation coefficient for respondents who watched 4-7 hours of screen time:", correlation)

## Analysis 3 | Rhemessa

# Step 2: Compare Screen Time of Deprived v. Non-Deprived
Is there a notable difference of screen time hours between deprived and non-deprived respondents?

a. Calculate screen time mean

In [62]:
# Gather screen time columns
screentime = ["C_we", "C_wk", "G_we", "G_wk", "S_we", "S_wk", "T_we", "T_wk"]

# Calculate mean
merged_data['overall_screentime'] = merged_data[screentime].mean(axis=1)

b. Compare screen time by deprivation status

In [63]:
# Assign labels for efficiency
merged_data['deprivation_label'] = merged_data['deprived'].map({0: 'non_dep', 1: 'dep'})

# Separate screen time by deprivation status
non_deprivation = merged_data[merged_data['deprivation_label'] == 'non_dep']['overall_screentime']
deprivation = merged_data[merged_data['deprivation_label'] == 'dep']['overall_screentime']

c. Calculate the mean based on deprivation status

In [ ]:
# Calculate the mean for non-deprived group
mean_non_dep = non_deprivation.mean()

# Calculate the mean for deprived group
mean_dep = deprivation.mean()

print("Mean Overall Screen Time for Non-Deprived:", mean_non_dep)
print("Mean Overall Screen Time for Deprived:", mean_dep)

d. Visualise the results

In [ ]:
# Bar plot for mean comparison
groups = ['Non-Deprived', 'Deprived']
means = [mean_non_dep, mean_dep]

plt.bar(groups, means, color=['#1f77b4', '#A3C1DA'])
plt.ylabel('Mean Overall Screen Time')
plt.title('Overall Screen Time for Non-Deprived vs Deprived Groups')
for i, mean in enumerate(means):
    plt.text(i, mean, f'{mean:.2f}', ha='center', va='bottom')
plt.ylim(0, max(means) + 1) 
plt.show()

# Histogram to show overall screen time
hist_data = merged_data[['deprivation_label', 'overall_screentime']]

plt.figure(figsize=(10, 6))

sns.histplot(data=hist_data, x='overall_screentime', hue='deprivation_label', bins=7,
             alpha=0.7, multiple='dodge', palette={'dep': '#A3C1DA', 'non_dep': '#1f77b4'})

plt.title('Overall Screen Time by Hours')
plt.xlabel('Overall Screen Time')
plt.ylabel('Frequency')

plt.show()

* Note: The second histogram shows the data is skewed right, therefore it is not normally distributed

e. Perform Mann-Whitney U test
* To understand if the difference in screen time were statistically significant, I opted to run the Mann-Whitney U test on the data. Because the data is not normally distributed it required a non-parametric test, thus the decision of the Mann-Whitney U.

In [ ]:
stat, p_value = mannwhitneyu(deprivation, non_deprivation, alternative='two-sided')

print(f'Mann-Whitney U statistic: {stat}')
print(f'P-value: {p_value}')

if p_value < 0.05:
    print('There is statistically signficant difference. Reject null hypothesis.')
else:
    print('There is no statistical significant difference between groups. Accept null hypothesis.')

## Conclusion of Step 2
 Data indicates a difference in overall screen time. Respondents from deprived areas reported higher screentime than respondents in non-deprivation areas. The significance of difference in screen time was futher supported by the result of the Mann-Whitney U test where the null hypothesis was rejected. Additionally, deprived groups reported more hours on devices at the higher range (4, 5, 6 hours) than non-deprived groups who reported spending more hours on the lower range (1, 2, 3 hours). 

# Step 3: Compare the Well-being Scores of Deprived v. Non-deprived Respondents
Will the difference in screen time between deprived and non-deprived respondents result in a meaningful difference in well-being outcomes?

a. Calculate and merge well-being mean

In [67]:
# List of well-being columns
wellbeing_cols = ['Optm', 'Usef', 'Relx', 'Intp', 'Engs', 'Dealpr', 
                  'Thcklr', 'Goodme', 'Clsep', 'Conf', 'Mkmind', 
                  'Loved', 'Intthg', 'Cheer']

# Calculating the overall well-being
merged_data['Overall_Wellbeing'] = merged_data[wellbeing_cols].mean(axis=1)

a. Compare well-being scores by deprivation status

In [68]:
# Assign labels for efficiency
merged_data['dep_status'] = merged_data['deprived'].map({0: 'non_dep', 1: 'dep'})

# Separate well-being scores by deprivation
non_depriv = merged_data[merged_data['dep_status'] == 'non_dep']['Overall_Wellbeing']
depriv = merged_data[merged_data['dep_status'] == 'dep']['Overall_Wellbeing']


b. Calculate mean based on deprivation status

In [ ]:
# Calculate the mean for non-deprived group
mean_non_deprived = non_depriv.mean()

# Calculate the mean for deprived group
mean_deprived = depriv.mean()

print("Mean Overall Wellbeing for Non-Deprived:", mean_non_deprived)
print("Mean Overall Wellbeing for Deprived:", mean_deprived)

c. Visualise the results

In [ ]:
# Histogram for overall well-being
plt.figure(figsize=(10, 6))
sns.histplot(data=merged_data, x='Overall_Wellbeing', hue='dep_status', multiple="dodge", bins=15,
             palette={'dep': '#A3C1DA', 'non_dep': '#1f77b4'})

plt.title('Overall Wellbeing by Deprivation Status')
plt.xlabel('Overall Wellbeing Score')
plt.ylabel('Frequency')

plt.show()


# Bar plot for mean well-being
groups = ['Non-Deprived', 'Deprived']
means = [mean_non_deprived, mean_deprived]

plt.bar(groups, means, color=['#1f77b4', '#A3C1DA'])
plt.ylabel('Mean Well-being Screen Time')
plt.title('Overall Well-being for Non-Deprived vs Deprived Groups')
for i, mean in enumerate(means):
    plt.text(i, mean, f'{mean:.2f}', ha='center', va='bottom')
plt.ylim(0, max(means) + 1) 
plt.show()

* Note: Histogram shows data is skewed left, therefore not normally distributed.

d. Perform Mann-Whitney U test

In [ ]:
stat, p_value = mannwhitneyu(depriv, non_depriv, alternative='two-sided')

print(f'Mann-Whitney U statistic: {stat}')
print(f'P-value: {p_value}')

if p_value < 0.05:
    print('There is a statistically signficant difference. Reject null hypothesis.')
else:
    print('There is no statistical significance in the difference between groups. Accept null hypothesis.')

## Conclusion of Step 3:
* Note: The well-being scores based on deprivation status indicate that respondents from areas with high deprivation reported slightly lower well-being scores. The statistical significance is confirmed by the non-parametric Mann-Whitney U test. The histogram is skewed left, indicating the data is not normally distributed.

# Step 4: Analysing the Distribution of the Results
Confirming how the data is distributed.

a. Checking screen time data distribution

In [ ]:
sample = merged_data['overall_screentime'].sample(5000, random_state=1)

stat, p_value = shapiro(sample)

if p_value < 0.05:
    print("Data is not normally distributed")
    print(p_value)

b. Checking well-being data distribution

In [ ]:
sample = merged_data['Overall_Wellbeing'].sample(5000, random_state=1)

stat, p_value = shapiro(sample)

if p_value < 0.05:
    print("Data is not normally distributed")
    print(p_value)

## Step 4 Conclusion:
As indicated by the relevant histograms, the data for both well-being scores and screen time is confirmed to be not normally distributed.

# Step 5: Isolating and Transforming the Data for Deprived Respondents
Since the data required for our predictive modelling is not normally distributed, it may help to transform it. Transforming data that is not normally distributed can pull it as close to normal distribution as possible. This can provide better results from our predictive modelling.

a. Creating a DataFrame copy to isolate values of deprived respondents

In [ ]:
# Isolate and store deprivation data in variable
isolated_dep = merged_data[merged_data['deprived'] == 1]

n = len(isolated_dep)
print("The sample size is: %d" % n)

print(isolated_dep.columns)

# Creating a new data frame with the required values
iso_df = isolated_dep.copy()

iso_df['iso_wb'] = isolated_dep[['Optm', 'Usef', 'Relx', 'Intp', 'Engs', 'Dealpr', 
                  'Thcklr', 'Goodme', 'Clsep', 'Conf', 'Mkmind', 
                  'Loved', 'Intthg', 'Cheer']].mean(axis=1)

iso_df['iso_st'] = isolated_dep[["C_we", "C_wk", "G_we", "G_wk", "S_we", "S_wk", "T_we", "T_wk"]].mean(axis=1)

b. Verifying the sample size and means against previous results to ensure we are working with the correct data

In [ ]:
# Verify the sample size 
n = len(iso_df)
print("The sample size is: %d" % n)

# Verify wellbeing mean
mean_wb = iso_df['iso_wb'].mean()
print(mean_wb)
# Verify screentime mean
mean_st = iso_df['iso_st'].mean()
print(mean_st)



c. Transforming data, re-evaluating, and visualising data distributions before and after log and square root transformations
* Note: Re-evaluating and visualising the data assists in deciding which data set (original data, log transformed, square root transformed) is closest to normal distribution. This will indicate which data will provide the best outcomes from the predictive model.

c.1. Transformation and visualisation of screen time data

* For the screen time data, a log transformation may work best as the data is skewed right and the log transformation will pull the data right. This can assist in distributing the data normally.

LOG TRANSFORMATION

In [ ]:
# Log transformation screen time
iso_df['log_iso_st'] = np.log(iso_df['iso_st'] + 1)

# Re-evaluating distribution (decided to use a smaller sample size for this test for accuracy)
sample = iso_df['log_iso_st'].sample(5000, random_state=1)
stat, p_value = shapiro(sample)
if p_value < 0.05:
    print("Data is not normally distributed")
    print(p_value)

SQUARE ROOT TRANSFORMATION

In [ ]:
# Square root transformation screen time
iso_df['sqrt_iso_st'] = np.sqrt(iso_df['iso_st'])

# Re-evaluating distribution (decided to use a smaller sample size for this test for accuracy)
sample = iso_df['sqrt_iso_st'].sample(5000, random_state=1)
stat, p_value = shapiro(sample)
if p_value < 0.05:
    print("Data is not normally distributed")
    print(p_value)

In [ ]:
##### ORIGINAL SCREEN TIME DATA VISUALISATION #####
sns.histplot(iso_df['iso_st'], bins=14, kde=True)

plt.xlabel('Screen Time')
plt.ylabel('Frequency')
plt.title('Original Screen Time Data')


plt.show()

In [ ]:
##### LOG TRANSFORMED SCREEN TIME DATA VISUALISATION #####
sns.histplot(iso_df['log_iso_st'], bins=14, kde=True)

plt.xlabel('Screen Time')
plt.ylabel('Frequency')
plt.title('Log Screen Time')


plt.show()

In [ ]:
##### SQUARE ROOT TRANSFORMED SCREEN TIME DATA VISUALISATION #####
sns.histplot(iso_df['sqrt_iso_st'], bins=14, kde=True)


plt.xlabel('Screen Time')
plt.ylabel('Frequency')
plt.title('SqRt Screen Time')


plt.show()

c.2. Transformation and visualisation of well-being data

SQUARE ROOT TRANSFORMATION

In [ ]:
# Square root transformation well-being
iso_df['sqrt_iso_wb'] = np.sqrt(iso_df['iso_wb'])

# Re-evaluating distribution (decided to use a smaller sample size for this test for accuracy)
sample = iso_df['sqrt_iso_wb'].sample(5000, random_state=1)
stat, p_value = shapiro(sample)
if p_value < 0.05:
    print("Data is not normally distributed")
    print(p_value)

LOG TRANSFORMATION

In [ ]:
# Log transformation well-being
iso_df['log_iso_wb'] = np.log(iso_df['iso_wb'] + 1)

# Re-evaluating distribution (decided to use a smaller sample size for this test for accuracy)
sample = iso_df['log_iso_wb'].sample(5000, random_state=1)
stat, p_value = shapiro(sample)
if p_value < 0.05:
    print("Data is not normally distributed")
    print(p_value)

In [ ]:
##### ORIGINAL WELL-BEING DATA VISUALISATION #####
sns.histplot(iso_df['iso_wb'], bins=15, kde=True)

plt.xlabel('Well-being')
plt.ylabel('Frequency')
plt.title('Original Well-being')


plt.show()

In [ ]:
##### SQUARE ROOT TRANSFORMED WELLBEING DATA VISUALISATION #####
sns.histplot(iso_df['sqrt_iso_wb'], bins=15, kde=True)

plt.xlabel('Well-being')
plt.ylabel('Frequency')
plt.title('SqRt Well-being')


plt.show()

In [ ]:
##### LOG TRANSFORMED WELLBEING DATA VISUALISATION #####
sns.histplot(iso_df['log_iso_wb'], bins=15, kde=True)

plt.xlabel('Well-being')
plt.ylabel('Frequency')
plt.title('Log Well-being')


plt.show()

## Step 5 Conclusion: 
Based on the p-value results of the Shapiro-Wilks test after performing the transformations, the log transformations had the best outcome in distribution. Therefore, the log transformations will work best for the predictive model. 

# Step 6: Predictive Model

a. Generate and analyse correlation matrix
* I chose to generate a correlation matrix to futher my understanding of the relationship between the two variables I wanted to isolate for my predictive model. The correlation matrix is a simple method that provides insight into the relationship between the two variables, how they interact with each other, and what trend/pattern they produce.

In [ ]:
# Calculate the correlation matrix for the specified columns
correlation_matrix = iso_df[['log_iso_st', 'log_iso_wb']].corr()

# Display the correlation matrix
print(correlation_matrix)

* Note: The results of the correlation matrix show little to no correlation, linear or otherwise, between my two variables. Therefore, a linear regression model will not fit my analysis. For the purpose of a predictive model, a tree model will work best for the data.

b. Generate scatter plot for futher visualisation

In [ ]:
sns.lmplot(x='log_iso_wb', y='log_iso_st', data=iso_df, height=6, aspect=1.25, scatter_kws={'s': 2.5})
plt.xlabel('Well-being')
plt.ylabel('Screen Time')
plt.show()

c. Generate Decision Tree Regressor model

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

X = iso_df[['log_iso_wb']]
y = iso_df['log_iso_st']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=30)

dt = DecisionTreeRegressor(max_depth=10, min_samples_split=5, random_state=5)
dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

# test the model
mse = mean_squared_error(y_test, y_pred)
print(mse)
sqrt_mse = np.sqrt(mean_squared_error(y_test, y_pred))
print(sqrt_mse)
r2 = r2_score(y_pred, y_test)
print(r2)

cross_val_score(dt, X_train, y_train, cv=10)

d. Generate scatterplot to visualise predictions of Decision Tree Regressor

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='#A3C1DA', alpha=0.6, edgecolors='#1f77b4')
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--') 
plt.title('Actual vs Predicted')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.grid(True)
plt.xlim(y.min(), y.max())
plt.ylim(y.min(), y.max())
plt.show()

* Note: Visualisation suggests the model is not learning effectively to provide meaningful predictions.

## Conclusion of Step 6:
After running the correlation matrix and finding little to no correlation between my data, I opted to use a tree model for my predictive model. Testing of the Decision Tree Regressor model produced result that were less than favourable in terms of predicting outcomes. This can be due to many reasons such as irrelevant data, overfitting or underfitting, and the feature engineering process. For more accurate predicitions there is potential to further analyse or optimise the data for better predictive outcomes. 

## Analysis 4 | Jabed

4. Exploring Screen Time Activities
#Calculate average screen time for each activity (Gaming, Smartphone, TV, Computer).

In [ ]:
# Calculate the mean screen time for each activity (combining weekday and weekend)
smartphone_mean = dataset2[['S_we', 'S_wk']].mean(axis=1).mean()
tv_mean = dataset2[['T_we', 'T_wk']].mean(axis=1).mean()
computer_mean = dataset2[['C_we', 'C_wk']].mean(axis=1).mean()
gaming_mean = dataset2[['G_we', 'G_wk']].mean(axis=1).mean()

# Print the results
print(f'Smartphone Mean: {smartphone_mean} hours')
print(f'TV Mean: {tv_mean} hours')
print(f'Computer Mean: {computer_mean} hours')
print(f'Gaming Mean: {gaming_mean} hours')

# Create histograms for each activity (weekend and weekday combined)
activities = {
    'Smartphone': dataset2[['S_we', 'S_wk']].mean(axis=1),
    'TV': dataset2[['T_we', 'T_wk']].mean(axis=1),
    'Computer': dataset2[['C_we', 'C_wk']].mean(axis=1),
    'Gaming': dataset2[['G_we', 'G_wk']].mean(axis=1)
}

# Plot histograms
for activity, data in activities.items():
    plt.figure()
    plt.hist(data, bins=20, alpha=0.7, color='blue', edgecolor='black')
    plt.title(f'Histogram of {activity} Screen Time')
    plt.xlabel('Hours per Day')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

# Prepare the data for the box plots by combining weekday and weekend screen time for each activity
data = {
    'Smartphone': dataset2[['S_we', 'S_wk']].mean(axis=1),
    'TV': dataset2[['T_we', 'T_wk']].mean(axis=1),
    'Computer': dataset2[['C_we', 'C_wk']].mean(axis=1),
    'Gaming': dataset2[['G_we', 'G_wk']].mean(axis=1)
}

# Convert the dictionary to a DataFrame for easier plotting
df = pd.DataFrame(data)

# Create a box plot for each screen time activity
plt.figure(figsize=(10, 6))
df.boxplot()
plt.title('Box Plot of Screen Time Activities')
plt.ylabel('Hours per Day')
plt.grid(True)
plt.show()

# Focusing on meaningful scatter plots only
# Plot Total Screen Time vs Overall Well-being
merged_data['Total_Screen_Time'] = merged_data[['C_we', 'C_wk', 'G_we', 'G_wk', 'S_we', 'S_wk', 'T_we', 'T_wk']].sum(axis=1)

plt.figure(figsize=(8, 6))
plt.scatter(merged_data['Total_Screen_Time'], merged_data['Overall_Wellbeing'], alpha=0.5, color='blue')
plt.title('Total Screen Time vs Overall Well-being')
plt.xlabel('Total Screen Time (Hours per Week)')
plt.ylabel('Overall Well-being Score')
plt.grid(True)
plt.show()

In [ ]:
# Step 1: Calculate Average Screen Time for Each Activity
average_screen_time = merged_data[['C_wk', 'C_we', 'G_wk', 'G_we', 'S_wk', 'S_we', 'T_wk', 'T_we']].mean()
print("Average Screen Time for Each Activity:\n", average_screen_time)

# Step 2: Generate Descriptive Statistics for Each Activity
screen_time_stats = merged_data[['C_wk', 'C_we', 'G_wk', 'G_we', 'S_wk', 'S_we', 'T_wk', 'T_we']].describe()
print("Descriptive Statistics for Screen Time Activities:\n", screen_time_stats)


5.Calculate the Overall Well-being Score

In [92]:
# Calculated overall well-being score as the mean of well-being indicators
merged_data['Overall_Wellbeing'] = merged_data[['Optm', 'Usef', 'Relx', 'Intp', 'Engs', 'Dealpr', 'Thcklr', 'Goodme', 'Clsep', 'Conf', 'Mkmind', 'Loved', 'Intthg', 'Cheer']].mean(axis=1)


6.Data Preparation

In [93]:
# Features and target variable
X = merged_data[['T_we']]  # TV usage on weekends as the predictor
y = merged_data['Overall_Wellbeing']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Step 2: Build the Predictive Model

In [94]:
# Creating and training the linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions
y_pred = lr_model.predict(X_test)


## Step 3: Evaluate the Model

In [ ]:
# Calculating evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse:.4f}')
print(f'R^2 Score: {r2:.4f}')


## Step 4: Visualize the Results

In [ ]:
# Scatter plot for actual vs predicted well-being scores
plt.scatter(y_test, y_pred, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--', linewidth=2)
plt.xlabel('Actual Well-being Score')
plt.ylabel('Predicted Well-being Score')
plt.title('Actual vs Predicted Well-being Score')
plt.show()
